# Setup

In [69]:
import pandas as pd
import numpy as np
NP_TYPE = np.double

# Initial state

In [70]:
def check_state(S, I, R, N):
    assert (N == S + I + R).all() # Checks both value and shapes
    assert (S >= 0).all()
    assert (R >= 0).all()
    assert (I >= 0).all()
#     assert np.isclose(N.sum(), POP_SIZE)

from os import path
DATA_DIR = path.join('..', 'data')
N = pd.read_csv(path.join(DATA_DIR, 'borough_pop.csv'))['Population'].as_matrix()
I = np.array([100] + [0] * (len(N)-1), NP_TYPE)
S = N - I
R = np.array([0] * len(N), NP_TYPE)
POP_SIZE = N.sum()
state = (S, I, R, N)
check_state(*state)
print(state)
POP_SIZE

(array([234027., 299347., 233495., 352763., 270418., 294050., 345829.,
       198136., 265665., 243672., 216980., 212924.,  92526., 236622.,
       301971., 177580., 188167., 281120., 123820., 196704., 240495.,
       286447., 270435., 180116.,  86472., 248140.,  87096., 329966.,
       161893., 249805., 243366., 298118., 184394., 306924.,   7472.]), array([100.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.]), array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0.]), array([234127, 299347, 233495, 352763, 270418, 294050, 345829, 198136,
       265665, 243672, 216980, 212924,  92526, 236622, 301971, 177580,
       188167, 281120, 123820, 196704, 240495, 286447, 270435, 180116,
        86472, 248140,  87096, 

7947055

# Constants

In [71]:
BETA = NP_TYPE(0.5 / 24)
GAMMA = NP_TYPE((1/3) / 24)
assert np.isclose(BETA / GAMMA, 1.5)

# Main

In [72]:
def update_state(F, S, I, R, N):
    S_I_interaction = BETA * S * I * 1/N
    Fdash = F.sum(axis=1)
    Snew = -S_I_interaction + F.T.dot(S) - Fdash * S + S
    Inew = S_I_interaction + F.T.dot(I) - Fdash * I + (1-GAMMA) * I
    Rnew = GAMMA * I + F.T.dot(R) - Fdash * R + R
    Nnew = Snew + Inew + Rnew
    return (Snew, Inew, Rnew, Nnew)

In [73]:
def update_single_row(row, F, S, I, R, N):
    i = row
    Snew = -BETA * S[i] * I[i] / N[i] \
            + sum(F[j][i] * S[j] for j in range(0, len(N))) \
            - sum(F[i][j] * S[i] for j in range(0, len(N))) \
            + S[i]
    
    Inew = BETA * S[i] * I[i] / N[i] \
            + sum(F[j][i] * I[j] for j in range(0, len(N))) \
            - sum(F[i][j] * I[i] for j in range(0, len(N))) \
            - GAMMA * I[i] + I[i]
    
    Rnew = GAMMA * I[i] \
            + sum(F[j][i] * R[j] for j in range(0, len(N))) \
            - sum(F[i][j] * R[i] for j in range(0, len(N))) \
            + R[i]
    
    Nnew = Snew + Inew + Rnew
    return (Snew, Inew, Rnew, Nnew)

In [74]:
def check_F(F):
    assert F.shape == (len(N), len(N))
    assert (F.sum(axis=1) < 1).all()
    
def generate_F():
    def generate_row():
        rand_row = np.random.rand(len(N))
        for i in range(1, len(N)+10):
            row = rand_row / i
            if row.sum() < 1:
                return row
    return np.array([generate_row() for _ in range(len(N))])

In [76]:
np.seterr(all='raise')
# F = np.ones((len(N),len(N)), NP_TYPE) / (len(N) + 3)
F = generate_F()
check_F(F)
STR_INDEX = ('S', 'I', 'R', 'N')
for step in range(1000):
    new_state = update_state(F, *state)
    check_state(*new_state)
    for i in range(len(N)):
        new_row_state = update_single_row(i, F, *state)
        for j in range(4):
            error = None
            try:
                assert np.isclose(new_state[j][i], new_row_state[j])
            except AssertionError:
                error = 'fail'
            except FloatingPointError:
                error = 'underflow'
            if error:
                print('A {} occured checking {} of row {}: matrix = {}, row wise = {}'\
                     .format(error, STR_INDEX[j], i, new_state[j][i], new_row_state[j]))
    state = new_state
print('Done')

Done


In [80]:
def update_row_by_row(F, state):
    for i in range(len(N)):
        new_row_state = update_single_row(i, F, *state)

In [81]:
%timeit update_row_by_row(F, state)

9.12 ms ± 559 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [83]:
%timeit update_state(F, *state)

43.6 µs ± 7.15 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)
